In [11]:
import pandas as pd
import json
import gc
import regex as re

#### If the category is "resource", answer types are ontology classes from either the DBpedia ontology or the Wikidata ontology. If category is "literal", answer types are either "number", "date", or "string". "boolean" answer type. If the category is "boolean", answer type is always "boolean".

In [2]:
dbpedia_json = None
wikidata_json = None

with open('../datasets/DBpedia/multiling_reverse_dbpedia.json') as json_file:
    dbpedia_json = json.load(json_file)
    
with open('../datasets/Wikidata/lcquad2_anstype_wikidata_train.json') as json_file:
    wikidata_json = json.load(json_file)

In [3]:
len_list = list()

for i in dbpedia_json:
    len_list.append(len(i['type']))

In [4]:
import numpy as np
np.percentile(len_list, 95)

5.0

In [5]:
dbpedia_json[0]

{'id': 'dbpedia_1177',
 'question': 'Was Jacqueline Kennedy Onassis a follower of Melkite Greek Catholic Church?',
 'category': 'boolean',
 'type': ['boolean'],
 'question_de': 'War Jacqueline Kennedy Onassis eine Anhängerin der melkitisch-griechisch-katholischen Kirche?',
 'question_fr': 'Jacqueline Kennedy Onassis était-elle une adepte de l&#39;Église catholique grecque melkite?',
 'question_es': '¿Jacqueline Kennedy Onassis fue seguidora de la Iglesia greco-católica melquita?',
 'question_ru': 'Была ли Жаклин Кеннеди Онассис последователем мелькитской греко-католической церкви?',
 'question_r_de': 'Was Jacqueline Kennedy Onassis a supporter of the Melkite Greek Catholic Church?',
 'question_r_ru': 'Was Jacqueline Kennedy Onassis a follower of the Melkite Greek Catholic Church?'}

In [6]:
wikidata_json[0]

{'id': 19719,
 'question': 'What periodical literature does Delta Air Lines use as a moutpiece?',
 'category': 'resource',
 'type': ['publication',
  'recurring',
  'intellectual work',
  'text',
  'communication medium',
  'serial']}

In [9]:
def make_dataframe(ds_json):
    ds_dict = { "id": list(), "question": list(), "question_de": list(), "question_fr": list(),
               "question_es": list(), "question_ru": list(), "question_r_de" : list(), "question_r_ru" : list(),
               "category": list(), 
               "type_1": list(), "type_2": list(), "type_3": list(), "type_4": list(), "type_5": list()}
    
    for item in ds_json:
        if item['question'] == None:
            continue
            
        id_ = item['id']
        question = item['question']
        question_r_de = item['question_r_de']
        question_r_ru = item['question_r_ru']
        question_de = item['question_de']
        question_fr = item['question_fr']
        question_es = item['question_es']
        question_ru = item['question_ru']
        
        category = item['category']

        if len(item['type']) > 6:
            type_1 = item['type'][6]
            type_2 = item['type'][5]
            type_3 = item['type'][4]
            type_4 = item['type'][3]
            type_5 = item['type'][2]
        elif len(item['type']) > 5:
            type_1 = item['type'][5]
            type_2 = item['type'][4]
            type_3 = item['type'][3]
            type_4 = item['type'][2]
            type_5 = item['type'][1]
        elif len(item['type']) > 4:
            type_1 = item['type'][4]
            type_2 = item['type'][3]
            type_3 = item['type'][2]
            type_4 = item['type'][1]
            type_5 = item['type'][0]
        elif len(item['type']) > 3:
            type_1 = item['type'][3]
            type_2 = item['type'][2]
            type_3 = item['type'][1]
            type_4 = item['type'][0]
            type_5 = "n/a"
        elif len(item['type']) > 2:
            type_1 = item['type'][2]
            type_2 = item['type'][1]
            type_3 = item['type'][0]
            type_4 = "n/a"
            type_5 = "n/a"
        elif len(item['type']) > 1:
            type_1 = item['type'][1]
            type_2 = item['type'][0]
            type_3 = "n/a"
            type_4 = "n/a"
            type_5 = "n/a"
        elif len(item['type']) > 0:
            type_1 = item['type'][0]
            type_2 = "n/a"
            type_3 = "n/a"
            type_4 = "n/a"
            type_5 = "n/a"
        else:
            type_1 = "n/a"
            type_2 = "n/a"
            type_3 = "n/a"
            type_4 = "n/a"
            type_5 = "n/a"
        
        ds_dict['id'].append(id_)
        
        ds_dict['question'].append(question)
        ds_dict['question_r_de'].append(question_r_de)
        ds_dict['question_r_ru'].append(question_r_ru)
        ds_dict['question_de'].append(question_de)
        ds_dict['question_fr'].append(question_fr)
        ds_dict['question_es'].append(question_es)
        ds_dict['question_ru'].append(question_ru)
        
        ds_dict['category'].append(category)
        ds_dict['type_1'].append(type_1)
        ds_dict['type_2'].append(type_2)
        ds_dict['type_3'].append(type_3)
        ds_dict['type_4'].append(type_4)
        ds_dict['type_5'].append(type_5)
    
    df = pd.DataFrame.from_dict(ds_dict)
    
    assert("dbo:Location" in df.type_1.values or "drug" in df.type_1.values)
    
    return df    
gc.collect()

0

In [10]:
dbpedia_df = make_dataframe(dbpedia_json)
#wikidata_df = make_dataframe(wikidata_json)

In [11]:
dbpedia_df.head()

,id,question,question_de,question_fr,question_es,question_ru,question_r_de,question_r_ru,category,type_1,type_2,type_3,type_4,type_5
0,dbpedia_1177,Was Jacqueline Kennedy Onassis a follower of M...,War Jacqueline Kennedy Onassis eine Anhängerin...,Jacqueline Kennedy Onassis était-elle une adep...,¿Jacqueline Kennedy Onassis fue seguidora de l...,Была ли Жаклин Кеннеди Онассис последователем ...,Was Jacqueline Kennedy Onassis a supporter of ...,Was Jacqueline Kennedy Onassis a follower of t...,boolean,boolean,n/a,n/a,n/a,n/a
1,dbpedia_14427,What is the name of the opera based on Twelfth...,"Wie heißt die Oper, die auf Twelfth Night basi...",Quel est le nom de l&#39;opéra basé sur Twelft...,¿Cuál es el nombre de la ópera basada en Noche...,Как называется опера по мотивам «Двенадцатой н...,What is the name of the opera based on Twelfth...,What is the name of the opera based on Twelfth...,resource,dbo:Opera,dbo:MusicalWork,dbo:Work,n/a,n/a
2,dbpedia_16615,When did Lena Horne receive the Grammy Award f...,Wann erhielt Lena Horne den Grammy Award für d...,Quand Lena Horne a-t-elle reçu le Grammy Award...,¿Cuándo recibió Lena Horne el premio Grammy al...,Когда Лена Хорн получила премию Грэмми за лучш...,When did Lena Horne receive the Grammy Award f...,When did Lena Horn receive the Grammy Award fo...,literal,date,n/a,n/a,n/a,n/a
3,dbpedia_23480,Do Prince Harry and Prince William have the sa...,Haben Prinz Harry und Prinz William die gleich...,Le prince Harry et le prince William ont-ils l...,¿El príncipe Harry y el príncipe William tiene...,У принца Гарри и принца Уильяма одни и те же р...,Do Prince Harry and Prince William have the sa...,Do Prince Harry and Prince William have the sa...,boolean,boolean,n/a,n/a,n/a,n/a
4,dbpedia_3681,What is the subsidiary company working for Leo...,"Was ist die Tochtergesellschaft, die für Leona...",Quelle est la filiale qui travaille pour Leona...,¿Cuál es la empresa subsidiaria que trabaja pa...,Какая дочерняя компания работает у Леонарда Ма...,What is the subsidiary that works for Leonard ...,Which subsidiary does Leonard Maltin have?,resource,dbo:EducationalInstitution,dbo:Organisation,dbo:Agent,n/a,n/a


In [11]:
wikidata_df.head()

,id,question,category,type_1,type_2,type_3,type_4,type_5
0,19719,What periodical literature does Delta Air Line...,resource,serial,communication medium,text,intellectual work,recurring
1,15554,Who is the child of Ranavalona I's husband?,resource,natural person,omnivore,person,n/a,n/a
2,974,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,boolean,n/a,n/a,n/a,n/a
3,27610,Which is the operating income for Qantas?,literal,number,n/a,n/a,n/a,n/a
4,24488,which cola starts with the letter p,resource,protected name,class,symbol,non-alcoholic beverage,carbonated beverage


In [12]:
dbpedia_df.category.value_counts()

resource    9584
literal     5188
boolean     2799
Name: category, dtype: int64

In [13]:
wikidata_df.category.value_counts()

resource    11683
literal      4429
boolean      2139
Name: category, dtype: int64

In [14]:
dbpedia_df.type_1.value_counts()

dbo:Agent                   4183
boolean                     2799
dbo:Location                2243
string                      2068
number                      1634
date                        1486
dbo:Work                     899
dbo:Award                    358
dbo:TopicalConcept           295
dbo:ChemicalSubstance        247
dbo:Activity                 204
dbo:Media                    188
dbo:Event                    115
dbo:Disease                  111
dbo:PersonFunction            98
dbo:Name                      91
dbo:Biomolecule               77
dbo:Species                   65
dbo:MeanOfTransportation      54
dbo:EthnicGroup               44
dbo:Country                   42
dbo:PoliticalParty            41
dbo:Language                  39
dbo:AnatomicalStructure       39
dbo:Device                    38
dbo:MedicalSpecialty          20
n/a                           16
dbo:UnitOfWork                15
dbo:Currency                  11
dbo:Food                      11
dbo:TimePe

In [15]:
wikidata_df.type_1.value_counts()

boolean                    2139
string                     2089
date                       1304
number                     1038
omnivore                    741
                           ... 
metallic material             1
final                         1
manifesto                     1
tower                         1
recurrent event edition       1
Name: type_1, Length: 1880, dtype: int64

In [11]:
dbpedia_df.to_csv("../datasets/dbpedia.csv", sep="|", index=False)
#wikidata_df.to_csv("../datasets/wikidata.csv", sep="|", index=False)

# Test data to csv

In [4]:
dbpedia_json = None

with open('../datasets/DBpedia/smarttask_dbpedia_test_questions.json') as json_file:
    dbpedia_json = json.load(json_file)

In [5]:
dbpedia_json[0:5]

[{'id': 'dbpedia_16015',
  'question': 'How many ingredients are in the grain} ?'},
 {'id': 'dbpedia_3885',
  'question': 'Is the case fatality rate of Fournier gangrene fewer than 9.0?'},
 {'id': 'dbpedia_12907',
  'question': 'Does the shelf life of spinach equal 8?'},
 {'id': 'dbpedia_7955',
  'question': 'What sound does a pig make in the French language?'},
 {'id': 'dbpedia_2376',
  'question': 'When was Fergie completed his record label in Interscope records?'}]

In [16]:
ds_dict = { "id": list(), "question": list()}

for item in dbpedia_json:
    id_ = item['id']
    
    if item['question'] == None:
        question = 'No Question'
        print(question)
    else:
        question = re.sub(r'[{}]', '', item['question'])
        
    ds_dict['id'].append(id_)
    ds_dict['question'].append(question)
    
df = pd.DataFrame.from_dict(ds_dict)

In [17]:
df.head()

,id,question
0,dbpedia_16015,How many ingredients are in the grain ?
1,dbpedia_3885,Is the case fatality rate of Fournier gangrene...
2,dbpedia_12907,Does the shelf life of spinach equal 8?
3,dbpedia_7955,What sound does a pig make in the French langu...
4,dbpedia_2376,When was Fergie completed his record label in ...


In [18]:
df.to_csv("../datasets/dbpedia_test.csv", sep="|", index=False)